In [2]:
import numpy as np 
import os, sys
import scipy
import scipy.io 
#import nn


# --------------------------------------------
# Load data
# --------------------------------------------
seed = 3452
split = 2
np.random.seed(seed)
dataset = scipy.io.loadmat('/Users/nathalie.willems/Documents/AAAM/machine_learning/molecular_interactions/Molecular_Atomization_Energy_Predictions/nn-qm7/qm7.mat')

# --------------------------------------------
# Extract training data
# --------------------------------------------
P = dataset['P'][range(0,split)+range(split+1,5)].flatten()
X = dataset['X'][P]
T = dataset['T'][0,P]
